In [44]:
!pip install tensorflow_datasets

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=d463dab091e65d7f67fa250be9cf5b419397a028a0e3786c73e27a70cc278b24
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise


In [46]:
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\User\tensorflow_datasets\tf_flowers\3.0.1.incompleteQ2VYOU\tf_flowers-train.tfrecord*...:  …

Dataset tf_flowers downloaded and prepared to C:\Users\User\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


In [48]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

58889256/58889256 [==============================] - 8s 0us/step


In [49]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [53]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

In [54]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [56]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 59s 903ms/step - loss: 0.9184 - accuracy: 0.7114 - val_loss: 1.0899 - val_accuracy: 0.6615
Epoch 2/50
65/65 [==============================] - 58s 900ms/step - loss: 0.4199 - accuracy: 0.8511 - val_loss: 0.9910 - val_accuracy: 0.7082
Epoch 3/50
65/65 [==============================] - 58s 900ms/step - loss: 0.2224 - accuracy: 0.9226 - val_loss: 1.0781 - val_accuracy: 0.7354
Epoch 4/50
65/65 [==============================] - 59s 903ms/step - loss: 0.1074 - accuracy: 0.9664 - val_loss: 1.0761 - val_accuracy: 0.7296
Epoch 5/50
65/65 [==============================] - 58s 896ms/step - loss: 0.0597 - accuracy: 0.9849 - val_loss: 1.1996 - val_accuracy: 0.7198
Epoch 6/50
65/65 [==============================] - 59s 904ms/step - loss: 0.0218 - accuracy: 0.9981 - val_loss: 1.1595 - val_accuracy: 0.7374
Epoch 7/50
65/65 [==============================] - 59s 905ms/step - loss: 0.0110 - accuracy: 0.9995 - val_loss: 1.2134 - val_accuracy: 0.7296

In [58]:
test_loss, test_acc = model.evaluate(test_ds, test_labels)
print('Accuracy:', test_acc)

35/35 [==============================] - 24s 693ms/step - loss: 0.0111 - accuracy: 1.0000
Accuracy: 1.0
